In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlsp-padded/reshaped/x_train548.npy
/kaggle/input/mlsp-padded/reshaped/x_train224.npy
/kaggle/input/mlsp-padded/reshaped/x_train768.npy
/kaggle/input/mlsp-padded/reshaped/x_train607.npy
/kaggle/input/mlsp-padded/reshaped/x_train1.npy
/kaggle/input/mlsp-padded/reshaped/x_train60.npy
/kaggle/input/mlsp-padded/reshaped/x_train945.npy
/kaggle/input/mlsp-padded/reshaped/x_train158.npy
/kaggle/input/mlsp-padded/reshaped/x_train706.npy
/kaggle/input/mlsp-padded/reshaped/x_train428.npy
/kaggle/input/mlsp-padded/reshaped/x_train229.npy
/kaggle/input/mlsp-padded/reshaped/x_train305.npy
/kaggle/input/mlsp-padded/reshaped/x_train578.npy
/kaggle/input/mlsp-padded/reshaped/x_train990.npy
/kaggle/input/mlsp-padded/reshaped/x_train950.npy
/kaggle/input/mlsp-padded/reshaped/x_train366.npy
/kaggle/input/mlsp-padded/reshaped/x_train605.npy
/kaggle/input/mlsp-padded/reshaped/x_train854.npy
/kaggle/input/mlsp-padded/reshaped/x_train620.npy
/kaggle/input/mlsp-padded/reshaped/x_train290.npy
/ka

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import json
import librosa
    
def get_data_features(n_mfcc):
    lst_x = []
    lst_y = list(np.load("../input/mlsp-padded/reshaped/y_train.npy"))
    with open("../input/mlsp-padded/reshaped/Labels_dict") as f:
        dict1 = json.loads(f.read())
    for m in range(1000):
        z = np.load("../input/mlsp-padded/reshaped/x_train" + str(m) + ".npy")
        features = librosa.feature.mfcc(S=librosa.power_to_db(z), n_mfcc = 16)
        lst_x.append(features)
    return lst_x, lst_y, dict1

def train_val_split(lst1, lst_y): # input shapes are: (1000, 128, 2584) and (1000, 10)
    count = 1
    lstx_train = []
    lstx_test = []
    lsty_train = []
    lsty_test = []
    for m in range(len(lst1)):
        if m <= (count*100) - 21:
            lstx_train.append(lst1[m])
            lsty_train.append(lst_y[m])
        else:
            lstx_test.append(lst1[m])
            lsty_test.append(lst_y[m])
        if m > (count*100) - 2:
            count = count + 1
    X_train = np.array(lstx_train)
    y_train = np.array(lsty_train)
    X_test = np.array(lstx_test)
    y_test =  np.array(lsty_test)
    del(lst1)
    x_train, y_train = shuffle(X_train, y_train)
    x_test, y_test = shuffle(X_test, y_test)
    x_train = x_train[..., np.newaxis]
    x_test = x_test[..., np.newaxis]
    return x_train, x_test, y_train, y_test

In [4]:
n_mfcc = 16
x_train, y_train, dict1 = get_data_features(n_mfcc)

In [5]:
x_train, x_test, y_train, y_test = train_val_split(x_train, y_train)

In [6]:
print(dict1)
(temp, f, t, num) = x_train.shape
print(f)
print(t)
print(num)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

{'Bark': 0, 'Meow': 1, 'Siren': 2, 'Shatter': 3, 'Knock': 4, 'Crying_and_sobbing': 5, 'Microwave_oven': 6, 'Vehicle_horn_and_car_horn_and_honking': 7, 'Doorbell': 8, 'Walk_and_footsteps': 9}
16
2584
1
(800, 16, 2584, 1)
(200, 16, 2584, 1)
(800, 10)
(200, 10)


In [34]:
N_classes = 10
def make_model(f, t, num):
    model1 = tf.keras.models.Sequential()
    model1.add(tf.keras.layers.Input(shape=(f, t, num)))
    model1.add(tf.keras.layers.Flatten())
    model1.add(tf.keras.layers.Dense(1024, activation='leaky_relu'))
    model1.add(tf.keras.layers.Dense(256, activation='leaky_relu'))
    model1.add(tf.keras.layers.Dense(64, activation='leaky_relu'))
    model1.add(tf.keras.layers.BatchNormalization())
    model1.add(tf.keras.layers.Dropout(0.5))
    model1.add(tf.keras.layers.Dense(N_classes, activation='softmax'))
    return model1
model1 = make_model(f, t, num)

In [35]:
model1.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 41344)             0         
_________________________________________________________________
dense_35 (Dense)             (None, 1024)              42337280  
_________________________________________________________________
dense_36 (Dense)             (None, 256)               262400    
_________________________________________________________________
dense_37 (Dense)             (None, 64)                16448     
_________________________________________________________________
batch_normalization_18 (Batc (None, 64)                256       
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 10)               

In [36]:
model1.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall()],
)

# Train model for 50 epochs, capture the history
history = model1.fit(x_train, y_train, verbose = 1, epochs=80, validation_data=(x_test, y_test))

Epoch 1/80
25/25 [==============================] - 1s 25ms/step - loss: 2.8530 - precision_9: 0.1453 - recall_9: 0.0312 - val_loss: 12.0828 - val_precision_9: 0.1117 - val_recall_9: 0.1100
Epoch 2/80
25/25 [==============================] - 0s 12ms/step - loss: 2.5932 - precision_9: 0.2671 - recall_9: 0.0487 - val_loss: 6.9174 - val_precision_9: 0.1217 - val_recall_9: 0.1150
Epoch 3/80
25/25 [==============================] - 0s 11ms/step - loss: 2.4360 - precision_9: 0.2945 - recall_9: 0.0538 - val_loss: 6.5941 - val_precision_9: 0.1405 - val_recall_9: 0.1300
Epoch 4/80
25/25 [==============================] - 0s 11ms/step - loss: 2.4604 - precision_9: 0.2671 - recall_9: 0.0538 - val_loss: 5.0312 - val_precision_9: 0.1348 - val_recall_9: 0.1200
Epoch 5/80
25/25 [==============================] - 0s 11ms/step - loss: 2.2948 - precision_9: 0.3425 - recall_9: 0.0625 - val_loss: 6.2324 - val_precision_9: 0.1322 - val_recall_9: 0.1150
Epoch 6/80
25/25 [==============================] - 0s

In [37]:
model1.save('model_4.h5')

In [38]:
yhat_probs = model1.predict(x_test, verbose=1)
# predict crisp classes for test set
#yhat_classes = model.predict_classes(x_test, verbose=1)
y_hat=np.argmax(yhat_probs,axis=1)
print(y_hat)
y_target = []
for m in y_test:
    r = np.where(m == 1)
    y_target.append(int(r[0]))
    #y_target.append()

7/7 [==============================] - 0s 3ms/step
[8 0 0 5 5 0 9 0 6 0 3 0 6 0 4 5 0 0 0 9 5 8 8 8 0 0 5 9 8 0 6 0 0 3 0 8 0
 0 0 0 0 6 0 0 8 3 0 8 2 0 0 0 0 3 4 0 6 6 0 0 2 0 6 0 0 0 5 6 6 6 0 0 0 0
 0 8 0 0 8 0 5 0 0 4 6 6 6 8 0 8 5 0 0 6 0 8 2 6 5 0 0 5 8 8 0 0 8 0 0 6 0
 3 0 6 0 5 6 8 0 8 7 0 6 0 0 8 3 0 0 3 0 8 3 0 0 8 4 0 0 2 0 0 2 5 0 0 0 0
 0 5 0 1 0 5 8 0 0 9 0 0 0 5 0 0 0 0 0 4 0 8 0 6 8 0 8 0 9 0 0 0 6 0 8 7 0
 0 5 0 0 0 0 0 8 0 8 0 3 0 0 3]


In [39]:
print(len(y_hat))
print(len(y_target))

200
200


In [40]:
from sklearn.metrics import classification_report
report = classification_report(y_target, y_hat, output_dict=True)
#report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()

In [41]:
lst = []
for m in range(10):
    for key, value in dict1.items():
        if value == m:
            #print(key)
            lst.append(key)
lst.append("accuracy")
lst.append("macro avg")
lst.append("weighted avg")
report_df["Labels"] = lst

In [42]:
report_df

,precision,recall,f1-score,support,Labels
0,0.168224,0.900,0.283465,20.000,Bark
1,1.000000,0.050,0.095238,20.000,Meow
2,1.000000,0.250,0.400000,20.000,Siren
3,0.900000,0.450,0.600000,20.000,Shatter
4,0.800000,0.200,0.320000,20.000,Knock
5,0.750000,0.600,0.666667,20.000,Crying_and_sobbing
6,0.428571,0.450,0.439024,20.000,Microwave_oven
7,0.500000,0.050,0.090909,20.000,Vehicle_horn_and_car_horn_and_honking
8,0.678571,0.950,0.791667,20.000,Doorbell
9,0.600000,0.150,0.240000,20.000,Walk_and_footsteps


In [43]:
report_df.to_csv("model_4_result.csv")